# Five Tribes of Machine Learning: Iris Classification
## An Educational Demonstration

Welcome! This notebook demonstrates how the five tribes of machine learning (from Pedro Domingos' book "The Master Algorithm") each approach the classic Iris flower classification problem.

### The Five Tribes:
- 🌳 **Symbolists** - Learn through logical rules
- 🧠 **Connectionists** - Learn by mimicking the brain
- 🧬 **Evolutionaries** - Learn through simulated evolution
- 📊 **Bayesians** - Learn through probabilistic inference
- 📏 **Analogizers** - Learn by recognizing similarity

### What You'll Learn:
1. How different ML paradigms approach the same problem
2. The philosophical differences between approaches
3. When to use each type of algorithm
4. Working implementations you can modify and experiment with

## Table of Contents
1. [Introduction](#introduction)
2. [Problem Setup](#problem-setup)
3. [🌳 Symbolists: Decision Trees](#symbolists)
4. [🧠 Connectionists: Neural Networks](#connectionists)
5. [🧬 Evolutionaries: Genetic Programming](#evolutionaries)
6. [📊 Bayesians: Naive Bayes](#bayesians)
7. [📏 Analogizers: k-Nearest Neighbors](#analogizers)
8. [Comparison & Conclusion](#comparison)
9. [Glossary](#glossary)